# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data_csv = 'output_data/cities.csv'
city_data = pd.read_csv(city_data_csv)
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Max Temperature,Latitude,Longitude,Humidity,Cloud %,Wind Speed
0,airai,75.15,-8.9266,125.4092,36,74,4.65
1,kavieng,83.39,-2.5744,150.7967,75,55,15.35
2,albany,68.67,42.6001,-73.9662,86,63,2.10
3,jerecuaro,57.99,20.1500,-100.5167,95,100,0.65
4,bantry,56.68,51.6833,-9.4500,90,100,19.39
...,...,...,...,...,...,...,...
558,fare,78.31,-16.7000,-151.0167,71,21,8.61
559,kenitra,70.56,34.2610,-6.5802,88,1,4.76
560,horizontina,60.26,-27.6258,-54.3078,74,100,4.12
561,preobrazheniye,79.50,42.9019,133.9064,78,100,3.89


In [4]:
#test to see if widgets are working since the map was not showing
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)


In [6]:
locations = city_data_df[['Latitude', 'Longitude']]
humidity = city_data_df['Humidity']
figure = gmaps.figure()


In [7]:
heatmap = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False,
                              max_intensity = 250, point_radius = 4)

figure.add_layer(heatmap)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#narrow max temp > 70 and < 80, wind speed less than 10, and zero cloudiness
narrowed_cities = city_data_df.loc[(city_data_df['Max Temperature'] < 80) & (city_data_df["Max Temperature"]
                                        > 70) & (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloud %'] == 0)]
print(len(narrowed_cities))
narrowed_cities


14


,City,Max Temperature,Latitude,Longitude,Humidity,Cloud %,Wind Speed
98,emerald,73.54,-23.5333,148.1667,25,0,8.05
138,mount isa,73.17,-20.7333,139.5000,16,0,9.22
143,panorama,76.42,40.5878,23.0315,38,0,1.01
154,labrea,72.43,-7.2586,-64.7981,51,0,0.31
161,ambilobe,73.35,-13.2000,49.0500,62,0,2.53
178,rodrigues alves,71.10,-7.7380,-72.6509,56,0,1.34
213,cayenne,77.09,4.9333,-52.3333,89,0,3.44
219,byron bay,73.51,-28.6500,153.6167,55,0,9.55
257,el dorado,73.92,38.7666,-120.5177,39,0,1.99
309,charters towers,75.61,-20.1000,146.2667,20,0,9.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:

vacation_locations = narrowed_cities[['City','Latitude', 'Longitude']]
vacation_lat = narrowed_cities['Latitude']
vacation_lon = narrowed_cities["Longitude"]
hotel_df = pd.DataFrame(narrowed_cities)
hotel_df["Hotel Name"] = ''

#this section is not working, I will try another approach. 
#Perhaps it is not working because of vacation_lat/lon not being iterated through.

#hotel_search_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={vacation_lat},{vacation_lon}&radius=5000&types=lodging&key={g_key}'
#for city in hotel_df.iterrows():
   # response = requests.get(hotel_search_url).json()
   # print(response)

In [11]:
#try using params
params = {
    'radius' : 5000,
    'types'  : 'lodging',
    'key'    : g_key
}

places_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

for place in hotel_df.iterrows():
    lat = place['Latitude']
    lon = place['Longitude']
    
    params['location'] = f'{lat},{lon}'
    hotel = requests.get(places_url, params=params).json()
    hotel_df.loc['Hotel Name'] = hotel['results']
    

{'html_attributions': [], 'next_page_token': 'Aap_uEChyqeptK_YTMohBoFvYTjZF_8Qfrdkd2PR5ZXGBGUIPh5psiMf_8WhYzXmlc9LfLujF84aDwiITpYjSniVZAA6s3J5k2Ouy11JXBV2L3Vh42M2RPpKce4kFuFpbsf3YJBB809wavBe8sMPGZFQhk5Wf8e_DUXuYOH6tOtL4gT-QESM8dG9_JR8zzw9s-smKdqgnqiADSIthpVt8hcTm--RMA6_-jEKMFXKWpdCqRQwUtzo4Mdg1ZyezAm0vPmPOz_FE6W2VqG7HdDq5tAFaS5ZDXpjOYLlMWRKPCosH3P7-vnEWfUSUWW_LHh1AI77LMSxpJ0pJFQK8EoWBEPQVPJN-NzlVh7P04t4wN1ryw9NAfciG4T9awv-7f7iLIWanrGvgmqOTW4BkGYIffi_K6yNeT0qAgr00IpG_w-XINxz-ZEkCY5NGyb1', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.5262125, 'lng': 148.1602479}, 'viewport': {'northeast': {'lat': -23.52486351970849, 'lng': 148.1615968802915}, 'southwest': {'lat': -23.5275614802915, 'lng': 148.1588989197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': 'The Irish Vil

{'html_attributions': [], 'next_page_token': 'Aap_uEDetmUbi-su8XXE6KNnRrXHKO5KLvtF8GsmfIZZgBg_tK_uc2gHm9WM4YPHm_2R8svlri5k8YQozVE2y13ESrDdhsd99qyDU2l-MQjhCjpUJlUAY7J6YgOIFM9jDq2Geb5lFOzZZiO6VzzkrZsIaiOIwNRnTx4wDr-JJBmDjfiE3m_hv8NtA4Skk2BnZ1wo6A_m_52GO_6Q6G7CygJ7YqDsJ-jGHNvZtmW1k_tEPo1kVJQ98YeMowZOF5JBIpszrrV9E4MllUPTIRQJEo_y6CGkIycId6oVuxfJSpC-tUfNJK49K42igl3mRZzuYkoCIka17nM7P7RCsCda6fE2xvQHvWDJA7G8cOAkHMnCYH64mkqL_WvyILrN7NgonFaNWhEl7Z7Pc_QKysvA8uJ5-NB39eKecla6UwdRLNgzUta3oF3ZWUT8yBA7', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -20.7259501, 'lng': 139.4925696}, 'viewport': {'northeast': {'lat': -20.7246348197085, 'lng': 139.4941328802915}, 'southwest': {'lat': -20.7273327802915, 'lng': 139.4914349197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'ibis Styles Mt Isa Ver

{'html_attributions': [], 'next_page_token': 'Aap_uEBXocRdGrhRnzQjxphmzkDKhxt8O__-z9ma2kz9WtgeeT8qTVb8XzdSarIhJlti0RFbEgVzlojfDxJYXNryfoLwqXDaWlxpk88d3BgIjN_a3Fa4iwgGBe0GP6y1_e2_hmRSadREqRkZwKZVuqwwYA4xP2T3pfcfwR_gqGa4J7eCKHdHefqhsO8Zv2Ca5unfBaVyrco9_mqmKbsMqSWlLHmjy_e4jLX3VHppTpjd7r8EcBQLKD23Wn7K3qrTcRJRJTEIa2xt4muYTOQXPKyoGk-zGX7WR7H0Fmqnk5sxN7lSrGwMH62BBNZTNU2m_-v4jGkLTHX-dMCVnNvT07gr2VkEOqEdUA9I_WMZZasqViZHMzrNvbT-jIc3h-hlYwLpinPuY95tbzKWvkPNc-UGA-8YSg63qRMfwEllguHru7PADebfnoVuBIQT', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 40.58901589999999, 'lng': 23.0298557}, 'viewport': {'northeast': {'lat': 40.5903515802915, 'lng': 23.0313385802915}, 'southwest': {'lat': 40.5876536197085, 'lng': 23.02864061970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Panorama Hotel', 'op

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -7.259713199999999, 'lng': -64.7963474}, 'viewport': {'northeast': {'lat': -7.258348519708497, 'lng': -64.79501341970848}, 'southwest': {'lat': -7.261046480291502, 'lng': -64.79771138029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hotel Novo Horizonte', 'photos': [{'height': 3000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102649009590112326200">Edson Alexandre Batista de Araújo Alexandre</a>'], 'photo_reference': 'Aap_uEBwc4cLmhhLOw78qKTjmUe-YwJQP2iP8DU-d8FXkznk1h-E73PaI6VsUlvAug_X-AyBKTmkmUTz66ewM6UtldhlK6Ys2V_U2y_qRp0bha7K5sKY92Ou7M1IlTdC5ylYalialRfFAGTsfiYm7Jfn2J9QHOBen__v6gWG-vFOtiYrmSss', 'width': 4000}], 'place_id': 'ChIJD6Bcf8WTL5IR64048J_MsdA', 'plus_code': {'compou

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -7.736708200000001, 'lng': -72.6478954}, 'viewport': {'northeast': {'lat': -7.735361469708498, 'lng': -72.6465883697085}, 'southwest': {'lat': -7.738059430291503, 'lng': -72.6492863302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Pousada Aconchego', 'photos': [{'height': 3000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117329486575768831831">Alef Soares</a>'], 'photo_reference': 'Aap_uEBaKdleyNjiTjvVTwu4rNQFqyBBeDRxWd5zCcR2WWOcT_CuuvLfeMH-d-uaXczHVUyas-HvPoAC5lu1VUBpnbvqcApihKPxxV3biWFdGfImDNC1OKpuyOUiMPpHSqC6fbDysfawUnFkqoQ4tEBIO0kuwHOldEh2miwCz7FWzgQtorpD', 'width': 4000}], 'place_id': 'ChIJ_VHNqenLmZERF81Jx01K4xc', 'plus_code': {'compound_code': '7972+8R Rodrigues Alves, A

{'html_attributions': [], 'next_page_token': 'Aap_uEBM8XscxXgnHw5SkCzhuYpOW9T0r-zTmLHpPrbdkBtfMi9ZOGd2lDoXPOeU2M-HbyG_YlnRxBzyKELhVy-U5ftEPBQeDLwg2UHz9OHRvtbewqOKCgzUmL1K3Fm1SANo7knflR8t5Hs55QGL3pj822SVOajPLdV9nbuRyhqrlQ-L9VhQQ-UnU-08rVYF2hVU0izHO7vPPbXQjLmNgvHx4lMJYkKFgpmOynS1qFlPvoRDULfcOo3YUX7N9qg1UJhhN_s3CSR2wydBGh8BtjXyh6_iknrqz70sFBr9ous3pXSMiPFnVn2t8Gd0jyy8SvFUTE0598_W_BygJv2J44K7ZspXh-iBhPHmDzaTm2QZW5e3RuOKxJPCAT4SKh96tqQ25Y4sfF4vmfOFRnR14xO8mSC-4KOuSG0wy3SGMxhgv5ZtbluRhFTxHHp4', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -28.6406707, 'lng': 153.6082656}, 'viewport': {'northeast': {'lat': -28.6394428697085, 'lng': 153.6095705802915}, 'southwest': {'lat': -28.6421408302915, 'lng': 153.6068726197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Backpackers Inn on the

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 38.760718, 'lng': -120.5205033}, 'viewport': {'northeast': {'lat': 38.7619333302915, 'lng': -120.5191949197085}, 'southwest': {'lat': 38.7592353697085, 'lng': -120.5218928802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/camping-71.png', 'icon_background_color': '#4DB546', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/camping_pinlet', 'name': 'South Fork Group Campground', 'place_id': 'ChIJH56Manw4moARJH4RPUyP2M4', 'plus_code': {'compound_code': 'QF6H+7Q Pollock Pines, CA, USA', 'global_code': '84CXQF6H+7Q'}, 'rating': 2, 'reference': 'ChIJH56Manw4moARJH4RPUyP2M4', 'scope': 'GOOGLE', 'types': ['campground', 'lodging', 'park', 'point_of_interest', 'establishment'], 'user_ratings_total': 1, 'vicinity': '7887 U.S. 50, Pollock Pines'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 38.7666163, 'lng': -120.

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -8.159988, 'lng': -70.76550089999999}, 'viewport': {'northeast': {'lat': -8.158453719708499, 'lng': -70.7641522697085}, 'southwest': {'lat': -8.161151680291502, 'lng': -70.76685023029151}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Flat Temporada', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107419132391157627835">Dalliana Morais</a>'], 'photo_reference': 'Aap_uEA664tKwh7_Y_d4aq3dKnARvpGoGRYkiFUObnNZkt_ztsVv5ZBPZnQW7G7VtGZSIa9wzXYhMfe8ytEl3Bx-Xq7i9lf-geR2Hqjufztg3y-4wNw5xIfclMd8adEEPCgYfbyjYRey4gDK5jr3eWDHyK0iNH16RUg4pk8acVaPA5WBkAN9', 'width': 4032}], 'place_id': 'ChIJW3cszOEim5ERHRpe4ne9Xuo', 'plus_code': {'compou

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 65.8517419, 'lng': 74.4880336}, 'viewport': {'northeast': {'lat': 65.8529672802915, 'lng': 74.4896544302915}, 'southwest': {'lat': 65.85026931970849, 'lng': 74.48695646970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': "Elizaveta Shevchuk's House", 'place_id': 'ChIJBavzB9dNYkMRVhIR2aZrPfY', 'plus_code': {'compound_code': 'VF2Q+M6 Pangody, Yamalo-Nenets Autonomous Okrug, Russia', 'global_code': '9JQPVF2Q+M6'}, 'reference': 'ChIJBavzB9dNYkMRVhIR2aZrPfY', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'vicinity': 'Ulitsa Mira, 15, Pangody'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 65.84546639999999, 'lng': 74.4714604}, 'viewport': {'northeast': {'l

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
